# Harmony Browse Image Service (HyBIG) regression tests

This Jupyter notebook runs a suite of regression tests against some requests against the Harmony Browse Image Service.

These tests use ASTER Global Digital Elevation Model (GDEM) Version 3 ([ASTGTM](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1256584478-EEDTEST)) as GeoTIFF input data to test the HyBIG service for an image with no color information.

These tests use MEaSUREs Vegetation Continuous Fields (VCF) Yearly Global 0.05 Deg V001 ([VCF5KYR](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1258119317-EEDTEST)) as GeoTIFF input data to test the HyBIG service against RGB color banded input.

## Set the Harmony environment:

The cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

The default value is for the UAT environment. When using this notebook there are two ways to use the non-default environment:

* Run this notebook in a local Jupyter notebook server and change the value of `harmony_host_url` in the cell below to the value for the environment you require from the above list.

* Use the `run_notebooks.sh` script, which requires you to declare an environment variable `HARMONY_HOST_URL`. Set that environment variable to the value above that corresponds to the environment you want to test. That environment variable will take precedence over the default value in the cell below.

In [ ]:
harmony_host_url = 'https://harmony.sit.earthdata.nasa.gov'

### Import required packages:

In [ ]:
from harmony import Collection, Environment, Client, Request
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS
from utility import  print_success 
from tempfile import TemporaryDirectory
from pathlib import Path
from numpy.testing import assert_array_almost_equal

### Set up environment dependent variables:

This includes the Harmony `Client` object and `Collection` objects for each of the collections for which there are regression tests. The local, SIT and UAT Harmony instances all utilise resources from CMR UAT, meaning any non-production environment will use the same resources.

When adding a production entry to the dictionary below, the collection instances can be included directly in the production dictionary entry, as they do not need to be shared.

In [ ]:
non_production_collection = {
    'aster_collection': Collection(id='C1256584478-EEDTEST'),
    'measures_collection': Collection(id='C1258119317-EEDTEST')
}

non_prod_granule_data = {
    'aster_granules': ['G1256584570-EEDTEST'],
    'measures_granules': ['G1258119387-EEDTEST']
}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.UAT
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.SIT
    },
    'http://localhost:3000': {
        **non_production_collection,
        **non_prod_granule_data,        
        'env': Environment.LOCAL
    },
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])
    endpoint_url = environment_information.get('endpoint_url', None)

### Test a request for PNG output from a 1-band GeoTIFF with no color information

Forms a request for ASTER data (1-band with no color info) as 'image/png', ensures all files are created that the image has the correct metadata and that that the data in the png file matches the reference data in the test.

In [ ]:
if environment_information is not None:
    aster_request = Request(collection=environment_information['aster_collection'],
                           granule_id=environment_information['aster_granules'][0],
                            format='image/png')                       
    aster_job_id = harmony_client.submit(aster_request)
    harmony_client.wait_for_processing(aster_job_id, show_progress=True)


    expected_files = ['ASTGTMV003_N00E022_dem.png',
                      'ASTGTMV003_N00E022_dem.pgw',
                      'ASTGTMV003_N00E022_dem.png.aux.xml']


    with TemporaryDirectory() as temp_dir:
        downloaded_grid_outputs = [
            file_future.result()
            for file_future
            in harmony_client.download_all(aster_job_id, overwrite=True, directory=temp_dir)
        ]
        
        
        reference_dir = Path('./regression_data')

        test_files = list(map(lambda fn: Path(temp_dir) / fn, expected_files))
        reference_files = list(map(lambda fn: reference_dir / fn, expected_files))

        for file_name in test_files:
            assert file_name.exists() == True, f'File does not exist {file_name.resolve()}'
        print_success('all test files generated')

        expected_output_metadata = {'driver': 'PNG',
                                    'dtype': 'uint8',
                                    'nodata': None,
                                    'width': 3601,
                                    'height': 3601,
                                    'count': 4,
                                    'crs': CRS.from_epsg(4326),
                                    'transform': Affine(0.000277777777777778, 0.0, 21.9998611111111,
                                                        0.0, -0.000277777777777778, 1.00013888888889)}

        test_dataset = rasterio.open(test_files[0])
        assert test_dataset.meta == expected_output_metadata, 'output PNG has incorrect metadata'
        print_success('Generated image has correct metadata.')

        reference_dataset = rasterio.open(reference_files[0])

        ref_image = reference_dataset.read()
        test_image = test_dataset.read()
        assert_array_almost_equal(ref_image, test_image)
        print_success('Generated image contains correct data.')


    print_success('Conversion of ASTER Geotiff to PNG Success')
else:
    print('Skipping test: HyBIG regression tests not configured for this environment.')

### Test a request for JPG output from a 1-band GeoTIFF with no color information

Forms a request for ASTER data as 'image/png', ensures all files are created that the image has the correct metadata and that that the data in the png file matches the reference data in the test.

*This test has to fix the downloaded aux.xml file becuase it is downloaded as gzipped but named as if it is not*

In [ ]:

if environment_information is not None:
    aster_request = Request(collection=environment_information['aster_collection'],
                           granule_id=environment_information['aster_granules'][0],
                            format='image/jpeg')      
    aster_job_id = harmony_client.submit(aster_request)
    harmony_client.wait_for_processing(aster_job_id, show_progress=True)


    expected_files = ['ASTGTMV003_N00E022_dem.jpg',
                      'ASTGTMV003_N00E022_dem.jgw',
                      'ASTGTMV003_N00E022_dem.jpg.aux.xml']


    with TemporaryDirectory() as temp_dir:
        downloaded_grid_outputs = [
            file_future.result()
            for file_future
            in harmony_client.download_all(aster_job_id, overwrite=True, directory=temp_dir)
        ]
     
        reference_dir = Path('./regression_data')

        test_files = list(map(lambda fn: Path(temp_dir) / fn, expected_files))
        reference_files = list(map(lambda fn: reference_dir / fn, expected_files))
        

        for file_name in test_files:
            assert file_name.exists() == True, f'File does not exist {file_name.resolve()}'
        print_success('all test files generated')

        expected_output_metadata = {'driver': 'JPEG',
                                    'dtype': 'uint8',
                                    'nodata': None,
                                    'width': 3601,
                                    'height': 3601,
                                    'count': 3,
                                    'crs': CRS.from_epsg(4326),
                                    'transform': Affine(0.000277777777777778, 0.0, 21.9998611111111,
                                                        0.0, -0.000277777777777778, 1.00013888888889)}

        test_dataset = rasterio.open(test_files[0])
        assert test_dataset.meta == expected_output_metadata, 'output JPEG has incorrect metadata'
        print_success('Generated image has correct metadata.')

        reference_dataset = rasterio.open(reference_files[0])

        ref_image = reference_dataset.read()
        test_image = test_dataset.read()
        assert_array_almost_equal(ref_image, test_image)
        print_success('Generated image contains correct data.')


    print_success('Conversion of ASTER Geotiff to JPEG Success')
else:
    print('Skipping test: HyBIG regression tests not configured for this environment.')

### Test a request for PNG output from 3-Band RGB input data

Forms a request for MEaSUREs VCF5KYR data (3-band RGB GeoTiff) as 'image/png', ensures all files are created that the image has the correct metadata and that that the data in the png file matches the reference data in the test.

In [ ]:
if environment_information is not None:
    measures_request = Request(collection=environment_information['measures_collection'],
                               granule_id=environment_information['measures_granules'][0],
                               format='image/png')                       

    measures_job_id = harmony_client.submit(measures_request)
    harmony_client.wait_for_processing(measures_job_id, show_progress=True)


    expected_files = ['VCF5KYR_1991001_001_2018224205008.png',
                      'VCF5KYR_1991001_001_2018224205008.pgw',
                      'VCF5KYR_1991001_001_2018224205008.png.aux.xml']


    with TemporaryDirectory() as temp_dir:
        downloaded_grid_outputs = [
            file_future.result()
            for file_future
            in harmony_client.download_all(measures_job_id, overwrite=True, directory=temp_dir)
        ]
        
        reference_dir = Path('./regression_data')

        test_files = list(map(lambda fn: Path(temp_dir) / fn, expected_files))
        reference_files = list(map(lambda fn: reference_dir / fn, expected_files))

        for file_name in test_files:
            assert file_name.exists() == True, f'File does not exist {file_name.resolve()}'
        print_success('all test files generated')

        expected_output_metadata = {
            'driver': 'PNG',
            'dtype': 'uint8',
            'nodata': None,
            'width': 7200,
            'height': 3600,
            'count': 3,
            'crs': CRS.from_wkt('GEOGCS["Unknown datum based upon the Clarke 1866 ellipsoid",'
                                'DATUM["Not_specified_based_on_Clarke_1866_spheroid",'
                                'SPHEROID["Clarke 1866",6378206.4,294.978698213898,'
                                'AUTHORITY["EPSG","7008"]]],'
                                'PRIMEM["Greenwich",0],'
                                'UNIT["degree",0.0174532925199433,'
                                'AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'),
            'transform': Affine(0.05000000000000001, 0.0, -180.0,
                                0.0, -0.05000000000000001, 90.0)
        }



        test_dataset = rasterio.open(test_files[0])
        assert test_dataset.meta == expected_output_metadata, 'output PNG has incorrect metadata'
        print_success('Generated image has correct metadata.')

        reference_dataset = rasterio.open(reference_files[0])

        ref_image = reference_dataset.read()
        test_image = test_dataset.read()
        assert_array_almost_equal(ref_image, test_image)
        print_success('Generated image contains correct data.')


    print_success('Conversion of MEaSUREs Geotiff to PNG Success')
else:
    print('Skipping test: HyBIG regression tests not configured for this environment.')

### Test a request for JPEG output from 3-Band RGB input data

Forms a request for MEaSUREs VCF5KYR data (3-band RGB GeoTiff) as 'image/jpg', ensures all files are created that the image has the correct metadata and that that the data in the JPEG file matches the reference data in the test.

In [ ]:
if environment_information is not None:
    measures_request = Request(collection=environment_information['measures_collection'],
                               granule_id=environment_information['measures_granules'][0],
                               format='image/jpeg')                       

    measures_job_id = harmony_client.submit(measures_request)
    harmony_client.wait_for_processing(measures_job_id, show_progress=True)


    expected_files = ['VCF5KYR_1991001_001_2018224205008.jpg',
                      'VCF5KYR_1991001_001_2018224205008.jgw',
                      'VCF5KYR_1991001_001_2018224205008.jpg.aux.xml']


    with TemporaryDirectory() as temp_dir:
        downloaded_grid_outputs = [
            file_future.result()
            for file_future
            in harmony_client.download_all(measures_job_id, overwrite=True, directory=temp_dir)
        ]
        
        reference_dir = Path('./regression_data')

        test_files = list(map(lambda fn: Path(temp_dir) / fn, expected_files))
        reference_files = list(map(lambda fn: reference_dir / fn, expected_files))

        for file_name in test_files:
            assert file_name.exists() == True, f'File does not exist {file_name.resolve()}'
        print_success('all test files generated')

        expected_output_metadata = {
            'driver': 'JPEG',
            'dtype': 'uint8',
            'nodata': None,
            'width': 7200,
            'height': 3600,
            'count': 3,
            'crs': CRS.from_wkt('GEOGCS["Unknown datum based upon the Clarke 1866 ellipsoid",'
                                'DATUM["Not_specified_based_on_Clarke_1866_spheroid",'
                                'SPHEROID["Clarke 1866",6378206.4,294.978698213898,'
                                'AUTHORITY["EPSG","7008"]]],'
                                'PRIMEM["Greenwich",0],'
                                'UNIT["degree",0.0174532925199433,'
                                'AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]'),
            'transform': Affine(0.05000000000000001, 0.0, -180.0,
                                0.0, -0.05000000000000001, 90.0)
        }



        test_dataset = rasterio.open(test_files[0])
        assert test_dataset.meta == expected_output_metadata, 'output JPEG has incorrect metadata'
        print_success('Generated image has correct metadata.')

        reference_dataset = rasterio.open(reference_files[0])

        ref_image = reference_dataset.read()
        test_image = test_dataset.read()
        assert_array_almost_equal(ref_image, test_image)
        print_success('Generated image contains correct data.')


    print_success('Conversion of MEaSUREs Geotiff to JPEG Success')
else:
    print('Skipping test: HyBIG regression tests not configured for this environment.')